#Chapter08
01

In [2]:
from enum import Enum

class IncomeSource(Enum):
    EMPLOYMENT = "employment"
    INDEPENDENT_CONTRACTOR = "independent_contractor"
    OFFICE = "office"
    PENSION = "pension"
    LUMP_SUM = "lump_sum"
    UNKNOWN = "unknown"

def classify_income(data: dict) -> IncomeSource:
    if data.get("is_director") is True:
        return IncomeSource.OFFICE
    if data.get("is_periodic_payment") is True:
        return IncomeSource.PENSION
    if data.get("is_lump_sum") is True:
        return IncomeSource.LUMP_SUM
    if data.get("contract_type") == "contract_for_service":
        return IncomeSource.INDEPENDENT_CONTRACTOR
    if data.get("contract_type") == "contract_of_service":
        return IncomeSource.EMPLOYMENT
    return IncomeSource.UNKNOWN


#Changeable_Fields01
- contract_type: "contract_of_service" or "contract_for_service"
- is_director: True / False
- is_periodic_payment: True / False
- is_lump_sum: True / False


In [3]:
sample_income = {
    "contract_type": "contract_of_service",
    "is_director": False,
    "is_periodic_payment": False,
    "is_lump_sum": False
}

classify_income(sample_income)


<IncomeSource.EMPLOYMENT: 'employment'>

02

In [4]:
def is_hong_kong_employment(data: dict) -> dict:
    contract_in_hk = data.get("contract_signed_in_hk", False)
    employer_resident_in_hk = data.get("employer_resident_in_hk", False)
    remuneration_paid_in_hk = data.get("remuneration_paid_in_hk", False)

    if contract_in_hk or employer_resident_in_hk or remuneration_paid_in_hk:
        return {
            "hk_source": True,
            "reason": "One of the DIPN 10 conditions is satisfied."
        }

    if data.get("remuneration_borne_by_hk_entity", False):
        return {
            "hk_source": True,
            "reason": "Remuneration borne by HK entity (totality of facts)."
        }

    return {
        "hk_source": False,
        "reason": "No sufficient HK connection under DIPN 10 or totality of facts."
    }


#Changeable_Fields02
- contract_signed_in_hk: True / False
- employer_resident_in_hk: True / False
- remuneration_paid_in_hk: True / False
- remuneration_borne_by_hk_entity: True / False


In [5]:
sample_employment = {
    "contract_signed_in_hk": False,
    "employer_resident_in_hk": False,
    "remuneration_paid_in_hk": False,
    "remuneration_borne_by_hk_entity": True
}

is_hong_kong_employment(sample_employment)


{'hk_source': True,
 'reason': 'Remuneration borne by HK entity (totality of facts).'}

03

#Changeable_Fields (in `employment_data`):
- "all_services_outside_hk": True / False
- "is_visitor": True / False
- "days_in_hk": integer (0–365)
- "is_researcher_or_teacher": True / False
- "is_dta_country": True / False
- "foreign_tax_paid": True / False

In [6]:
def check_hk_employment_taxability(data: dict) -> dict:
    """
    Determine whether a Hong Kong-sourced employment is exempt from taxation, or fully chargeable to Salaries Tax.
    """


    # Section 8(1A)(b)
    if data.get("all_services_outside_hk", False):
        return {
            "taxable": False,
            "reason": "Exempt under Section 8(1A)(b): all services rendered outside Hong Kong."
        }

    # Section 8(1B)
    if data.get("is_visitor", False) and data.get("days_in_hk", 999) <= 60:
        return {
            "taxable": False,
            "reason": "Exempt under Section 8(1B): visitor present in Hong Kong for 60 days or less."
        }

    # Section 8(1AB)
    if data.get("is_researcher_or_teacher", False) and data.get("is_dta_country", False) and data.get("foreign_tax_paid", False):
        return {
            "taxable": False,
            "reason": "Exempt under Section 8(1AB): researcher/teacher in DTA country and foreign tax was paid."
        }

    # Default: Fully taxable
    return {
        "taxable": True,
        "reason": "Employment is sourced in Hong Kong and no exemption conditions are satisfied."
    }


In [7]:
employment_data = {
    "all_services_outside_hk": False,
    "is_visitor": True,
    "days_in_hk": 55,
    "is_researcher_or_teacher": False,
    "is_dta_country": False,
    "foreign_tax_paid": False
}

check_hk_employment_taxability(employment_data)


{'taxable': False,
 'reason': 'Exempt under Section 8(1B): visitor present in Hong Kong for 60 days or less.'}

04

#Changeable_Fields (in `non_hk_employment_data`):
- `total_service_days`: total working days during the period (int)
- `hk_service_days`: working days physically in HK (int)
- `income_amount`: total employment income (float)
- `foreign_tax_paid`: True / False
- `dta_country`: True / False

In [8]:
def calculate_non_hk_employment_tax(data: dict) -> dict:
    """
    Perform time apportionment for non-Hong Kong sourced employment and determine whether the income is exempt or eligible for tax relief.
    """


    total_days = data.get("total_service_days", 365)
    hk_days = data.get("hk_service_days", 0)
    income = data.get("income_amount", 0)

    if total_days == 0:
        return {
            "taxable": False,
            "amount": 0,
            "reason": "Invalid service days (0)."
        }

    ratio = hk_days / total_days
    taxable_amount = round(income * ratio, 2)

    if data.get("foreign_tax_paid", False):
        if data.get("dta_country", False):
            return {
                "taxable": True,
                "amount": taxable_amount,
                "relief": "Tax credit available under Section 50 (DTA country).",
                "reason": f"{ratio:.2%} of income is attributable to Hong Kong."
            }
        else:
            return {
                "taxable": False,
                "amount": 0,
                "relief": "Excluded under Section 8(1A)(c): foreign tax paid, no DTA.",
                "reason": "Income excluded as foreign tax was paid and no DTA exists."
            }

    return {
        "taxable": True,
        "amount": taxable_amount,
        "relief": None,
        "reason": f"{ratio:.2%} of income is attributable to Hong Kong based on time apportionment."
    }


In [9]:
non_hk_employment_data = {
    "total_service_days": 300,
    "hk_service_days": 90,
    "income_amount": 600000,
    "foreign_tax_paid": True,
    "dta_country": False
}

calculate_non_hk_employment_tax(non_hk_employment_data)


{'taxable': False,
 'amount': 0,
 'relief': 'Excluded under Section 8(1A)(c): foreign tax paid, no DTA.',
 'reason': 'Income excluded as foreign tax was paid and no DTA exists.'}

05

#Changeable_Fields (in `visitor_data`):
- `days_in_hk_current_year`: int
- `days_in_hk_last_year`: int
- `is_aircrew_or_seafarer`: True / False
- `is_mainland_resident`: True / False
- `employer_is_mainland_entity`: True / False

In [10]:
def check_visitor_exemption(data: dict) -> dict:
    """
    Determine whether exemption can be granted based on visitor status.
    """

    current_year_days = data.get("days_in_hk_current_year", 365)
    last_year_days = data.get("days_in_hk_last_year", 0)
    total_days = current_year_days + last_year_days

    # Section 8(1B)
    if current_year_days <= 60:
        return {
            "exempt": True,
            "reason": "Exempt under Section 8(1B): visitor present in Hong Kong for no more than 60 days."
        }

    # Section 8(2)(j)
    if data.get("is_aircrew_or_seafarer", False) and current_year_days <= 60 and total_days <= 120:
        return {
            "exempt": True,
            "reason": "Exempt under Section 8(2)(j): seafarer/aircrew within 60+120 day limit."
        }

    # Mainland DTA 183-day rule
    if data.get("is_mainland_resident", False) and data.get("employer_is_mainland_entity", False) and current_year_days <= 183:
        return {
            "exempt": True,
            "reason": "Exempt under PRC-HK DTA: mainland resident working for mainland entity, ≤183 days in HK."
        }

    return {
        "exempt": False,
        "reason": "Visitor exemption does not apply under current facts."
    }


In [11]:
visitor_data = {
    "days_in_hk_current_year": 59,
    "days_in_hk_last_year": 50,
    "is_aircrew_or_seafarer": False,
    "is_mainland_resident": False,
    "employer_is_mainland_entity": False
}

check_visitor_exemption(visitor_data)


{'exempt': True,
 'reason': 'Exempt under Section 8(1B): visitor present in Hong Kong for no more than 60 days.'}

06

#Changeable_Fields (in `office_income_data`):
- `central_management_location`: "Hong Kong", "Mainland", "Singapore", etc.

In [12]:
def is_office_income_sourced_in_hk(data: dict) -> dict:
    """
    Determine whether directors’ fees or office-holder income are sourced in Hong Kong.
    """


    location = data.get("central_management_location", "").lower()

    if location == "hong kong":
        return {
            "hk_source": True,
            "reason": "Company is centrally managed and controlled in Hong Kong."
        }
    elif location:
        return {
            "hk_source": False,
            "reason": f"Company's central management and control is located in {location.title()}."
        }
    else:
        return {
            "hk_source": False,
            "reason": "Central management location not specified."
        }


In [13]:
office_income_data = {
    "central_management_location": "Hong Kong"
}

is_office_income_sourced_in_hk(office_income_data)


{'hk_source': True,
 'reason': 'Company is centrally managed and controlled in Hong Kong.'}

07

#Changeable_Fields (in `pension_data`):
- `pension_fund_location`: "Hong Kong", "UK", etc.
- `total_service_years`: int
- `hk_service_years`: int
- `pension_amount`: float

In [14]:
def calculate_taxable_pension(data: dict) -> dict:
    """
    Determine whether the pension is sourced in Hong Kong and calculate the chargeable amount based on the proportion of years of service.
    """


    location = data.get("pension_fund_location", "").lower()
    if location != "hong kong":
        return {
            "taxable": False,
            "amount": 0,
            "reason": f"Pension fund is managed outside Hong Kong ({location.title()})."
        }

    total_years = data.get("total_service_years", 1)
    hk_years = data.get("hk_service_years", 0)
    pension = data.get("pension_amount", 0)

    if total_years <= 0:
        return {
            "taxable": False,
            "amount": 0,
            "reason": "Invalid total service years."
        }

    ratio = hk_years / total_years
    taxable_amount = round(pension * ratio, 2)

    return {
        "taxable": True,
        "amount": taxable_amount,
        "reason": f"{ratio:.2%} of pension is attributable to Hong Kong services."
    }


In [15]:
pension_data = {
    "pension_fund_location": "Hong Kong",
    "total_service_years": 30,
    "hk_service_years": 18,
    "pension_amount": 300000
}

calculate_taxable_pension(pension_data)


{'taxable': True,
 'amount': 180000.0,
 'reason': '60.00% of pension is attributable to Hong Kong services.'}

08

#Changeable_Fields (in `relief_data`):
- `foreign_tax_paid`: True / False
- `has_dta`: True / False
- `hk_tax_amount`: float
- `foreign_tax_amount`: float

In [16]:
def apply_foreign_tax_relief(data: dict) -> dict:
    """
    Determine whether foreign tax relief is available.
    """


    if not data.get("foreign_tax_paid", False):
        return {
            "relief": None,
            "taxable": True,
            "net_tax": data.get("hk_tax_amount", 0),
            "reason": "No foreign tax paid. Full HK tax applies."
        }

    if not data.get("has_dta", False):
        return {
            "relief": "income exclusion",
            "taxable": False,
            "net_tax": 0,
            "reason": "Foreign tax paid in a non-DTA country. Income excluded under Section 8(1A)(c)."
        }

    # If DTA exists, apply tax credit
    hk_tax = data.get("hk_tax_amount", 0)
    foreign_tax = data.get("foreign_tax_amount", 0)
    credit = min(hk_tax, foreign_tax)
    net_tax = max(0, hk_tax - credit)

    return {
        "relief": "tax credit",
        "taxable": True,
        "net_tax": net_tax,
        "reason": f"Tax credit of {credit} applied under Section 50. Net HK tax: {net_tax}."
    }


In [17]:
relief_data = {
    "foreign_tax_paid": True,
    "has_dta": True,
    "hk_tax_amount": 25000,
    "foreign_tax_amount": 18000
}

apply_foreign_tax_relief(relief_data)


{'relief': 'tax credit',
 'taxable': True,
 'net_tax': 7000,
 'reason': 'Tax credit of 18000 applied under Section 50. Net HK tax: 7000.'}

#Chapter09
01

#Changeable_Fields (in `general_income_items`):
Each income item should be a dictionary with:
- `type`: a keyword string representing the type of income  
  Valid taxable types include:
  - "salary"
  - "bonus"
  - "commission"
  - "gratuity"
  - "leave_pay"
  - "allowance"
  - "perquisite"
  - "holiday_benefit"
  - "education_benefit"
- `amount`: a number indicating the HKD amount

In [18]:
def assess_general_income(income_items: list) -> dict:
    """
    Evaluate and summarize general remuneration items under Section 9(1).
    """


    chargeable_types = {
        "salary", "bonus", "commission", "gratuity", "leave_pay", "allowance",
        "perquisite", "holiday_benefit", "education_benefit"
    }

    total_taxable = 0
    details = []

    for item in income_items:
        item_type = item.get("type", "").lower()
        amount = item.get("amount", 0)

        if item_type in chargeable_types:
            total_taxable += amount
            details.append(f"{item_type}: HKD {amount} (chargeable)")
        else:
            details.append(f"{item_type}: HKD {amount} (not chargeable by S9(1))")

    return {
        "taxable_amount": total_taxable,
        "breakdown": details
    }


In [19]:
general_income_items = [
    {"type": "salary", "amount": 400000},
    {"type": "bonus", "amount": 50000},
    {"type": "education_benefit", "amount": 30000},
    {"type": "transport_subsidy", "amount": 10000}  # 假设未列于 S9(1)
]

assess_general_income(general_income_items)


{'taxable_amount': 480000,
 'breakdown': ['salary: HKD 400000 (chargeable)',
  'bonus: HKD 50000 (chargeable)',
  'education_benefit: HKD 30000 (chargeable)',
  'transport_subsidy: HKD 10000 (not chargeable by S9(1))']}

02

#Changeable_Fields (in `lump_sum_data`):
- `amount`: float – total lump sum received
- `is_contractual`: True / False
- `is_reward_for_service`: True / False
- `is_loss_of_office`: True / False


In [20]:
def assess_lump_sum_income(data: dict) -> dict:
    """
    Determine whether a lump-sum payment is chargeable to tax.
    """


    amount = data.get("amount", 0)
    is_contractual = data.get("is_contractual", False)
    is_reward = data.get("is_reward_for_service", False)
    is_loss_of_office = data.get("is_loss_of_office", False)

    if is_contractual:
        return {
            "taxable": True,
            "taxable_amount": amount,
            "reason": "Taxable under Section 9 as it is contractually stipulated."
        }

    if is_reward:
        return {
            "taxable": True,
            "taxable_amount": amount,
            "reason": "Taxable under Section 9 as it is a reward for past/present/future service."
        }

    if is_loss_of_office:
        return {
            "taxable": False,
            "taxable_amount": 0,
            "reason": "Exempt – payment is compensation for loss of office (not employment income)."
        }

    return {
        "taxable": False,
        "taxable_amount": 0,
        "reason": "Payment does not fall under contractual or service-related category; treated as non-taxable by default."
    }

In [21]:
lump_sum_data = {
    "amount": 300000,
    "is_contractual": False,
    "is_reward_for_service": False,
    "is_loss_of_office": True
}

assess_lump_sum_income(lump_sum_data)


{'taxable': False,
 'taxable_amount': 0,
 'reason': 'Exempt – payment is compensation for loss of office (not employment income).'}

03

#Changeable_Fields (in `housing_data`):
- `employment_income`: float – total income from employer
- `housing_type`: "flat", "hotel_1_room", "hotel_suite"
- `employee_paid_rent`: float
- `refund_to_employer`: float

In [22]:
def calculate_housing_benefit(data: dict) -> dict:
    """
    Calculate the Notional Rental Value (NRV) and apply applicable deductions.
    """


    income = data.get("employment_income", 0)
    housing_type = data.get("housing_type", "flat")
    paid_rent = data.get("employee_paid_rent", 0)
    refund = data.get("refund_to_employer", 0)

    rate_dict = {
        "flat": 0.10,
        "hotel_1_room": 0.04,
        "hotel_suite": 0.08
    }

    rate = rate_dict.get(housing_type, 0.10)
    base_nrv = round(income * rate, 2)
    deduction = min(paid_rent, refund)
    final_taxable = max(0, base_nrv - deduction)

    return {
        "taxable": True,
        "taxable_amount": final_taxable,
        "base_nrv": base_nrv,
        "deducted": deduction,
        "reason": f"NRV = {rate*100:.0f}% of income ({base_nrv}) - deduction ({deduction}) = {final_taxable}"
    }


In [23]:
housing_data = {
    "employment_income": 600000,
    "housing_type": "flat",
    "employee_paid_rent": 20000,
    "refund_to_employer": 15000
}

calculate_housing_benefit(housing_data)


{'taxable': True,
 'taxable_amount': 45000.0,
 'base_nrv': 60000.0,
 'deducted': 15000,
 'reason': 'NRV = 10% of income (60000.0) - deduction (15000) = 45000.0'}

04

#Changeable_Fields (in `option_data`):
- `exercise_type`: "exercise", "assignment", "release"
- `market_value`: float – value of share on trigger event date
- `exercise_price`: float – strike price of option
- `option_cost`: float – any payment made to acquire the option
- `use_apportionment`: True / False
- `total_service_months`: int – vesting period
- `hk_service_months`: int – months of Hong Kong-related service


In [24]:
def calculate_share_option_gain(data: dict) -> dict:
    """
    Calculate the taxable gain from stock options based on the type of exercise and whether the gain is apportioned.
    """


    mv = data.get("market_value", 0)
    ep = data.get("exercise_price", 0)
    cost = data.get("option_cost", 0)
    exercise_type = data.get("exercise_type", "exercise")
    apportion = data.get("use_apportionment", False)
    total_months = data.get("total_service_months", 1)
    hk_months = data.get("hk_service_months", 0)

    if exercise_type == "exercise":
        gain = mv - ep - cost
    elif exercise_type in {"assignment", "release"}:
        gain = mv - cost
    else:
        return {
            "taxable": False,
            "reason": "Invalid exercise type. Must be exercise, assignment, or release."
        }

    gain = round(max(gain, 0), 2)

    if apportion and total_months > 0:
        ratio = hk_months / total_months
        taxable_gain = round(gain * ratio, 2)
        reason = f"Apportioned: {ratio:.2%} of gain is taxable = {taxable_gain}."
    else:
        taxable_gain = gain
        reason = f"Full gain is taxable: {gain}."

    return {
        "taxable": True,
        "taxable_amount": taxable_gain,
        "gain_before_apportionment": gain,
        "reason": reason
    }


In [25]:
option_data = {
    "exercise_type": "exercise",
    "market_value": 150,
    "exercise_price": 100,
    "option_cost": 5,
    "use_apportionment": True,
    "total_service_months": 24,
    "hk_service_months": 12
}

calculate_share_option_gain(option_data)


{'taxable': True,
 'taxable_amount': 22.5,
 'gain_before_apportionment': 45,
 'reason': 'Apportioned: 50.00% of gain is taxable = 22.5.'}

05

#Changeable Fields (in `award_data`):
- `award_type`: "upfront", "backend", or "phantom"
- `market_value`: float – value of shares awarded
- `vesting_years`: int (used only for upfront awards, default 5)
- `years_elapsed`: int – how many years completed (used for upfront)

In [26]:
def calculate_share_award_tax(data: dict) -> dict:
    """
    Determine the taxable amount of share awards based on the type of award and the length of service.
    """


    award_type = data.get("award_type", "backend")
    mv = data.get("market_value", 0)
    vesting_years = data.get("vesting_years", 5)
    years_elapsed = data.get("years_elapsed", vesting_years)

    if award_type == "phantom":
        return {
            "taxable": True,
            "taxable_amount": mv,
            "reason": "Phantom share is fully taxable as cash bonus equivalent."
        }

    elif award_type == "backend":
        return {
            "taxable": True,
            "taxable_amount": mv,
            "reason": "Backend share is fully taxable upon grant (vesting condition met)."
        }

    elif award_type == "upfront":
        if vesting_years <= 0:
            return {"taxable": False, "taxable_amount": 0, "reason": "Invalid vesting period."}
        portion = min(years_elapsed / vesting_years, 1)
        taxable = round(mv * portion, 2)
        return {
            "taxable": True,
            "taxable_amount": taxable,
            "reason": f"Upfront share apportioned: {portion:.0%} of value taxable = {taxable}."
        }

    else:
        return {"taxable": False, "taxable_amount": 0, "reason": "Invalid award type."}


In [27]:
award_data = {
    "award_type": "upfront",
    "market_value": 100000,
    "vesting_years": 5,
    "years_elapsed": 3
}

calculate_share_award_tax(award_data)


{'taxable': True,
 'taxable_amount': 60000.0,
 'reason': 'Upfront share apportioned: 60% of value taxable = 60000.0.'}

06

#Changeable_Fields (in `pension_data`):
- `amount`: float – total retirement payment received
- `is_rors`: True / False – is it from a Recognized Occupational Retirement Scheme?
- `qualifies_for_exemption`: True / False – due to retirement, death, or disability?
- `employer_contribution_ratio`: float – e.g. 0.15 for 15%

In [28]:
def assess_retirement_payment(data: dict) -> dict:
    """
    Determine whether retirement income is subject to tax.
    """


    amount = data.get("amount", 0)
    is_rors = data.get("is_rors", False)
    qualifies = data.get("qualifies_for_exemption", False)
    employer_ratio = data.get("employer_contribution_ratio", 0.0)

    if is_rors and qualifies:
        return {
            "taxable": False,
            "taxable_amount": 0,
            "reason": "Exempt – received from RORS upon retirement/death/disability."
        }

    if is_rors and not qualifies:
        if employer_ratio <= 0.15:
            return {
                "taxable": False,
                "taxable_amount": 0,
                "reason": "RORS but non-qualifying, and employer contribution within 15% – no taxable portion."
            }
        else:
            excess_ratio = employer_ratio - 0.15
            taxable = round(amount * excess_ratio, 2)
            return {
                "taxable": True,
                "taxable_amount": taxable,
                "reason": f"Non-qualifying RORS: employer contribution exceeded 15%. Taxable portion = {excess_ratio:.0%} of total = {taxable}."
            }

    return {
        "taxable": True,
        "taxable_amount": amount,
        "reason": "Non-RORS retirement payment is fully taxable under S9(1)(aa)."
    }


In [29]:
pension_data = {
    "amount": 300000,
    "is_rors": True,
    "qualifies_for_exemption": False,
    "employer_contribution_ratio": 0.22
}

assess_retirement_payment(pension_data)


{'taxable': True,
 'taxable_amount': 21000.0,
 'reason': 'Non-qualifying RORS: employer contribution exceeded 15%. Taxable portion = 7% of total = 21000.0.'}

07

#Changeable_Fields (in `benefit_data`):
- `benefit_type`: e.g. "education", "vacation", "debt_payment", "gift"
- `benefit_value`: float
- `liability_test`: True / False – Is this the employee’s original liability?
- `convertibility_test`: True / False – Can it be converted to cash?

In [30]:
def assess_fringe_benefit(data: dict) -> dict:
    """
    Determine whether a fringe benefit is subject to tax.
    """


    benefit_type = data.get("benefit_type", "other")
    value = data.get("benefit_value", 0)
    liability = data.get("liability_test", False)
    convertibility = data.get("convertibility_test", False)

    if liability or convertibility:
        return {
            "taxable": True,
            "taxable_amount": value,
            "reason": f"Fringe benefit '{benefit_type}' is taxable because it satisfies the {'liability' if liability else 'convertibility'} test."
        }

    return {
        "taxable": False,
        "taxable_amount": 0,
        "reason": f"Fringe benefit '{benefit_type}' does not satisfy either tax test – not taxable."
    }


In [31]:
benefit_data = {
    "benefit_type": "education",
    "benefit_value": 30000,
    "liability_test": True,
    "convertibility_test": False
}

assess_fringe_benefit(benefit_data)


{'taxable': True,
 'taxable_amount': 30000,
 'reason': "Fringe benefit 'education' is taxable because it satisfies the liability test."}

08

#hangeable Fields (in `non_taxable_data`):
- `income_type`: e.g. "retirement_scheme", "maintenance", "gift"
- `amount`: float

In [32]:
def check_non_taxable_income(data: dict) -> dict:
    """
    判断是否属于 Section 8(2) 下免税收入。
    """

    income_type = data.get("income_type", "").lower()
    amount = data.get("amount", 0)

    exempt_types = {
        "maintenance",           
        "retirement_scheme",    
        "gift",                  
        "scholarship",          
        "inheritance"            
    }

    if income_type in exempt_types:
        return {
            "taxable": False,
            "taxable_amount": 0,
            "reason": f"{income_type.title()} is exempt under Section 8(2) or not within salaries tax scope."
        }

    return {
        "taxable": True,
        "taxable_amount": amount,
        "reason": f"{income_type.title()} is not specifically exempt – considered taxable unless further analysis applies."
    }


In [33]:
non_taxable_data = {
    "income_type": "inheritance",
    "amount": 200000
}

check_non_taxable_income(non_taxable_data)


{'taxable': False,
 'taxable_amount': 0,
 'reason': 'Inheritance is exempt under Section 8(2) or not within salaries tax scope.'}

#Chapter10
01

#Changeable_Fields (in `general_deduction_data`):
- `expenditure_amount`: float – actual amount spent
- `is_necessary`: True / False – whether required to earn income
- `is_reimbursed`: True / False – whether paid back by employer
- `is_depreciable_item`: True / False – e.g. tools, equipment
- `depreciation_rate`: float – only used if depreciable, default 20%

In [34]:
def calculate_general_deduction(data: dict) -> dict:
    """
    Determine whether general expenses are deductible.
    """


    amount = data.get("expenditure_amount", 0)
    necessary = data.get("is_necessary", False)
    reimbursed = data.get("is_reimbursed", False)
    depreciable = data.get("is_depreciable_item", False)
    rate = data.get("depreciation_rate", 0.2)

    if not depreciable:
        if necessary and not reimbursed:
            return {
                "deductible": True,
                "deduction_amount": amount,
                "reason": "Necessary expenditure incurred in producing income, not reimbursed by employer (S12(1)(a))."
            }
        else:
            return {
                "deductible": False,
                "deduction_amount": 0,
                "reason": "Expenditure is either not necessary or reimbursed – not deductible."
            }

    depreciation = round(amount * rate, 2)
    if necessary and not reimbursed:
        return {
            "deductible": True,
            "deduction_amount": depreciation,
            "reason": f"Depreciation on income-generating asset allowed at {rate*100:.0f}% rate under S12(1)(b)."
        }
    else:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Depreciable asset not used for income generation or was reimbursed."
        }


In [35]:
general_deduction_data = {
    "expenditure_amount": 20000,
    "is_necessary": True,
    "is_reimbursed": False,
    "is_depreciable_item": True,
    "depreciation_rate": 0.2
}

calculate_general_deduction(general_deduction_data)


{'deductible': True,
 'deduction_amount': 4000.0,
 'reason': 'Depreciation on income-generating asset allowed at 20% rate under S12(1)(b).'}

02-08

In [36]:
def cap_deduction(amount, cap, reason_text):
    return {
        "deductible": True,
        "deduction_amount": min(amount, cap),
        "reason": f"{reason_text} (Capped at HKD {cap})"
    }

In [37]:
def assess_self_education_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    is_qualified = data.get("is_qualified_course", True)

    if not is_qualified:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Not a qualified course under Section 12(1)(e)."
        }

    return cap_deduction(amount, 100000, "Qualified self-education expenses deductible")


In [38]:
def assess_charity_donation_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    assessable_income = data.get("assessable_income", 0)
    is_approved_charity = data.get("is_approved_charity", True)

    if not is_approved_charity:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Donation not made to an approved charitable institution (S26C)."
        }

    cap = round(assessable_income * 0.35, 2)
    return cap_deduction(amount, cap, f"Charitable donation deductible up to 35% of assessable income ({cap})")


In [39]:
def assess_elderly_care_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    is_parent_or_grandparent = data.get("is_parent_or_grandparent", True)

    if not is_parent_or_grandparent:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Person being cared for is not a qualified relative under Section 26D."
        }

    return cap_deduction(amount, 100000, "Elderly residential care expenses deductible")


In [40]:
def assess_ars_expense_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    is_qualified = data.get("is_qualified", True)

    if not is_qualified:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "ARS expense is not qualified under Inland Revenue Rules."
        }

    return {
        "deductible": True,
        "deduction_amount": min(amount, 100000),
        "reason": f"ARS expense deductible (capped at 100000). Claimed: {min(amount, 100000)}"
    }


In [41]:
def assess_home_loan_interest_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    is_principal_residence = data.get("is_principal_residence", True)

    if not is_principal_residence:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Loan interest not related to principal residence – not deductible."
        }

    return cap_deduction(amount, 100000, "Home loan interest deductible")


In [42]:
def assess_retirement_contribution_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    is_mpf = data.get("is_mpf", True)

    if not is_mpf:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Contribution is not to a recognized MPF / ORSO scheme."
        }

    return cap_deduction(amount, 18000, "Mandatory retirement contribution deductible")


In [43]:
def assess_annuity_and_mpf_deduction(data: dict) -> dict:
    annuity = data.get("annuity_amount", 0)
    voluntary_mpf = data.get("voluntary_mpf_amount", 0)
    total = annuity + voluntary_mpf

    return cap_deduction(total, 60000, "Annuity premium + voluntary MPF deductible (combined cap)")


In [44]:
def assess_domestic_rent_deduction(data: dict) -> dict:
    amount = data.get("amount", 0)
    has_valid_lease = data.get("has_valid_lease", True)
    is_also_claiming_home_loan = data.get("is_also_claiming_home_loan", False)

    if not has_valid_lease:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "No valid tenancy agreement – rent deduction not allowed."
        }

    if is_also_claiming_home_loan:
        return {
            "deductible": False,
            "deduction_amount": 0,
            "reason": "Cannot claim rent deduction and home loan interest in the same year."
        }

    return cap_deduction(amount, 100000, "Domestic rent deductible under Section 26V")


In [45]:
{
    'deductible': True,
    'deduction_amount': 60000,
    'reason': 'Annuity premium + voluntary MPF deductible (combined cap)'
}


{'deductible': True,
 'deduction_amount': 60000,
 'reason': 'Annuity premium + voluntary MPF deductible (combined cap)'}

In [46]:
def calculate_total_deductions(data: dict) -> dict:
    results = {}
    total = 0

    edu = assess_self_education_deduction(data.get("self_education", {}))
    results["self_education"] = edu
    total += edu["deduction_amount"]

    ars = assess_ars_expense_deduction(data.get("ars_expense", {}))
    results["ars_expense"] = ars
    total += ars["deduction_amount"]

    mpf = assess_retirement_contribution_deduction(data.get("retirement", {}))
    results["retirement"] = mpf
    total += mpf["deduction_amount"]

    return {
        "total_deduction": total,
        "details": results
    }


#Chapter11
1-9

In [47]:
ALLOWANCES = {
    "basic": 132000,
    "married": 264000,
    "child": 120000,
    "newborn_additional": 130000,
    "dependent_parent": 50000,
    "co_resident_additional": 10000,
    "single_parent": 132000,
    "disabled_dependant": 75000,
    "personal_disability": 75000,
    "dependent_sibling": 37500
}


In [48]:
def assess_basic_allowance(data: dict) -> dict:
    if data.get("is_single", True):
        return {"granted": True, "amount": ALLOWANCES["basic"], "reason": "Basic allowance granted."}
    return {"granted": False, "amount": 0, "reason": "Not single."}

def assess_married_allowance(data: dict) -> dict:
    if data.get("is_married", False) and data.get("elect_joint_assessment", True):
        return {"granted": True, "amount": ALLOWANCES["married"], "reason": "Married person’s allowance granted."}
    return {"granted": False, "amount": 0, "reason": "Not eligible for married person’s allowance."}

def assess_single_parent_allowance(data: dict) -> dict:
    if data.get("is_single_parent", False):
        return {"granted": True, "amount": ALLOWANCES["single_parent"], "reason": "Single parent allowance granted."}
    return {"granted": False, "amount": 0, "reason": "Not eligible for single parent allowance."}

def assess_child_allowance(data: dict) -> dict:
    n = data.get("num_children", 0)
    nb = data.get("num_newborn", 0)
    total = n * ALLOWANCES["child"] + nb * ALLOWANCES["newborn_additional"]
    return {"granted": n > 0, "amount": total, "reason": f"{n} child(ren), {nb} newborn(s) – total calculated."}

def assess_dependent_parent_allowance(data: dict) -> dict:
    n = data.get("num_parents", 0)
    c = data.get("num_co_resident", 0)
    base = n * ALLOWANCES["dependent_parent"]
    extra = c * ALLOWANCES["co_resident_additional"]
    return {"granted": n > 0, "amount": base + extra, "reason": f"{n} parent(s), {c} co-residing."}

def assess_dependent_grandparent_allowance(data: dict) -> dict:
    n = data.get("num_grandparents", 0)
    c = data.get("num_co_resident", 0)
    base = n * ALLOWANCES["dependent_parent"]
    extra = c * ALLOWANCES["co_resident_additional"]
    return {"granted": n > 0, "amount": base + extra, "reason": f"{n} grandparent(s), {c} co-residing."}

def assess_dependent_sibling_allowance(data: dict) -> dict:
    n = data.get("num_siblings", 0)
    return {"granted": n > 0, "amount": n * ALLOWANCES["dependent_sibling"], "reason": f"{n} dependent sibling(s)."}

def assess_disabled_dependant_allowance(data: dict) -> dict:
    n = data.get("num_disabled_dependants", 0)
    return {"granted": n > 0, "amount": n * ALLOWANCES["disabled_dependant"], "reason": f"{n} disabled dependant(s)."}

def assess_personal_disability_allowance(data: dict) -> dict:
    if data.get("is_disabled", False):
        return {"granted": True, "amount": ALLOWANCES["personal_disability"], "reason": "Personal disability granted."}
    return {"granted": False, "amount": 0, "reason": "Not eligible."}


In [49]:
def calculate_total_allowances(data: dict) -> dict:
    results = {}
    total = 0

    basic = assess_basic_allowance(data)
    married = assess_married_allowance(data)
    single = assess_single_parent_allowance(data)
    child = assess_child_allowance(data)
    parent = assess_dependent_parent_allowance(data)
    grandparent = assess_dependent_grandparent_allowance(data)
    sibling = assess_dependent_sibling_allowance(data)
    disabled_dep = assess_disabled_dependant_allowance(data)
    personal_dis = assess_personal_disability_allowance(data)

    if basic["granted"] and married["granted"]:
        basic["granted"] = False
        basic["amount"] = 0
        basic["reason"] = "Excluded due to married person’s allowance."

    if married["granted"] and single["granted"]:
        single["granted"] = False
        single["amount"] = 0
        single["reason"] = "Single parent allowance not allowed with married status."

    results["basic"] = basic
    results["married"] = married
    results["single_parent"] = single
    results["child"] = child
    results["dependent_parent"] = parent
    results["dependent_grandparent"] = grandparent
    results["dependent_sibling"] = sibling
    results["disabled_dependant"] = disabled_dep
    results["personal_disability"] = personal_dis

    for module in results.values():
        total += module["amount"]

    return {"total_allowance": total, "details": results}


#supplement_01
NRV&Rateable value

In [50]:
def assess_housing_benefit(data: dict) -> dict:
    """
    Assess taxable housing benefit using the lower of Notional Rental Value (NRV) and Rateable Value,
    and deduct any rent suffered by the employee.

    Parameters:
    - income: annual income relevant to housing benefit
    - nrv_ratio: typically 0.1 (10%), 0.08 or 0.04 depending on housing type
    - rent_suffered: any rent paid by the employee
    - rateable_value: official rateable value if applicable (optional)

    Returns:
    - taxable_amount: amount to be taxed as housing benefit
    - reason: explanation for how value was selected
    """
    income = data.get("income", 0)
    nrv_ratio = data.get("nrv_ratio", 0.1)
    rent_paid = data.get("rent_suffered", 0)
    rateable_value = data.get("rateable_value", None)

    nrv = income * nrv_ratio
    selected_value = min(nrv, rateable_value) if rateable_value else nrv
    taxable_value = max(selected_value - rent_paid, 0)

    reason = (
        f"Taxable = min(NRV {nrv}, Rateable {rateable_value}) - rent paid {rent_paid}"
        if rateable_value
        else f"Taxable = NRV {nrv} - rent paid {rent_paid} (no rateable value provided)"
    )

    return {
        "taxable": True,
        "taxable_amount": round(taxable_value, 2),
        "reason": reason
    }


#supplement_02
PRC183DAYS

In [52]:
def assess_prc_183_day_exemption(data: dict) -> dict:
    """
    Determine whether income from PRC-sourced employment is exempt under 183-day rule
    in a DTA context, such as for PRC-based employment with limited presence in Hong Kong.

    Required fields in data:
    - is_prc_employment: True/False
    - days_in_hk: number of days present in Hong Kong in the year of assessment
    - paid_by_non_hk_employer: True/False
    - borne_by_non_hk_entity: True/False
    - prc_tax_paid: True/False

    Returns:
    - exempt: True/False
    - reason: explanatory string
    """
    if not data.get("is_prc_employment", False):
        return {
            "exempt": False,
            "reason": "Employment is not sourced in the PRC; rule not applicable."
        }

    days = data.get("days_in_hk", 0)
    paid_by_non_hk = data.get("paid_by_non_hk_employer", False)
    cost_not_borne_in_hk = data.get("borne_by_non_hk_entity", False)
    prc_tax_paid = data.get("prc_tax_paid", False)

    if days <= 183 and paid_by_non_hk and cost_not_borne_in_hk and prc_tax_paid:
        return {
            "exempt": True,
            "reason": "Exempt under DTA 183-day rule: <183 days, non-HK employer, cost not borne in HK, and PRC tax paid."
        }

    return {
        "exempt": False,
        "reason": "One or more exemption conditions not met under DTA 183-day rule."
    }


#supplement_03
TVC+QDAP

In [54]:
MAX_RETIREMENT_QUOTA = 60000  # TVC + QDAP 合并上限

def assess_retirement_products_deduction(data: dict) -> dict:
    """
    Assess total deductible amount for QDAP and TVC contributions under the combined cap of HK$60,000.

    Expected input format:
    {
        "qdap": {"amount": ..., "is_qualified": True/False},
        "tvc": {"amount": ..., "is_qualified": True/False}
    }

    Returns:
    {
        "qdap": {"deductible": True/False, "deduction_amount": ..., "reason": ...},
        "tvc":  {"deductible": True/False, "deduction_amount": ..., "reason": ...},
        "total_deduction": ...
    }
    """
    qdap_data = data.get("qdap", {})
    tvc_data = data.get("tvc", {})

    qdap_amt = qdap_data.get("amount", 0) if qdap_data.get("is_qualified", False) else 0
    tvc_amt = tvc_data.get("amount", 0) if tvc_data.get("is_qualified", False) else 0

    used_quota = 0
    qdap_used = min(qdap_amt, MAX_RETIREMENT_QUOTA)
    used_quota += qdap_used
    remaining = MAX_RETIREMENT_QUOTA - used_quota
    tvc_used = min(tvc_amt, remaining)

    result = {
        "qdap": {
            "deductible": qdap_used > 0,
            "deduction_amount": qdap_used,
            "reason": f"QDAP deduction allowed up to {qdap_used} (combined cap: 60,000)"
        },
        "tvc": {
            "deductible": tvc_used > 0,
            "deduction_amount": tvc_used,
            "reason": f"TVC deduction allowed up to {tvc_used} (remaining cap after QDAP)"
        },
        "total_deduction": qdap_used + tvc_used
    }
    return result


#supplement04
depreciation

In [55]:
def assess_depreciation_allowance(data: dict) -> dict:
    """
    Assess depreciation allowance for capital expenditure on plant/machinery essential to earning income.

    Expected input format:
    {
        "items": [
            {"description": "Laptop", "cost": 12000, "rate": 0.2, "business_use_ratio": 1.0},
            {"description": "Vehicle", "cost": 100000, "rate": 0.3, "business_use_ratio": 0.5}
        ]
    }

    Returns:
    {
        "deductible": True/False,
        "deduction_amount": float,
        "details": [...],
        "reason": "Summary..."
    }
    """
    items = data.get("items", [])
    total = 0
    details = []

    for item in items:
        cost = item.get("cost", 0)
        rate = item.get("rate", 0)
        ratio = item.get("business_use_ratio", 1.0)
        desc = item.get("description", "Unnamed item")

        if cost <= 0 or rate <= 0 or ratio <= 0:
            continue

        allowance = cost * rate * ratio
        total += allowance
        details.append({
            "item": desc,
            "cost": cost,
            "rate": rate,
            "business_use_ratio": ratio,
            "deduction": round(allowance, 2)
        })

    return {
        "deductible": total > 0,
        "deduction_amount": round(total, 2),
        "details": details,
        "reason": f"Total depreciation deduction for {len(details)} item(s): {round(total, 2)}"
    }


#suppplement05
brother&sister

In [56]:
def assess_dependent_sibling_allowance_with_conflict_check(data: dict) -> dict:
    """
    Assess dependent sibling allowance while checking for conflict with disabled dependant allowance (DDA).

    Expected input:
    {
        "num_siblings": 2,
        "disabled_sibling_indices": [0],  # indexes of siblings already claimed under DDA
        "claimed_as_disabled_dependants": 1  # total siblings also counted in DDA
    }

    Returns:
    {
        "granted": True/False,
        "amount": int,
        "reason": string
    }
    """
    ALLOWANCE_PER_SIBLING = 37500
    total = data.get("num_siblings", 0)
    overlap = data.get("claimed_as_disabled_dependants", 0)

    conflict_count = min(total, overlap)
    allowed_count = total - conflict_count
    amount = allowed_count * ALLOWANCE_PER_SIBLING

    reason = (
        f"{total} dependent sibling(s) declared, {conflict_count} conflict with DDA removed. "
        f"{allowed_count} eligible for allowance."
        if conflict_count > 0
        else f"All {total} sibling(s) eligible."
    )

    return {
        "granted": allowed_count > 0,
        "amount": amount,
        "reason": reason
    }


#supplement06
search_table

In [58]:
# 减税表
ONE_OFF_TAX_REBATE = {
    2017: 20000,
    2018: 20000,
    2019: 20000,
    2020: 20000,
    2021: 10000,
    2022: 10000,
    2023: 6000,
    2024: 1500
}

# 函数本体
def apply_one_off_rebate(year: int, tax_payable: float) -> dict:
    rebate_cap = ONE_OFF_TAX_REBATE.get(year, 0)
    rebate = min(tax_payable, rebate_cap)
    final = tax_payable - rebate

    return {
        "rebate_applied": round(rebate, 2),
        "final_tax": round(final, 2),
        "reason": f"One-off tax rebate of ${rebate} applied (cap: ${rebate_cap}) for YOA {year}."
        if rebate_cap > 0 else
        f"No one-off tax rebate available for YOA {year}."
    }


In [59]:
def calculate_total_income(income_items: list) -> dict:
    """
    Aggregate all assessable income items using corresponding assess_xxx_income functions.

    Each item in income_items should include:
    - type: one of ['salary', 'bonus', 'housing', 'pension', 'stock_option', 'share_award', 'lump_sum']
    - other fields required by the specific assess_xxx function

    Returns:
    {
        "total_income": float,
        "details": list of dicts per item
    }
    """
    total = 0
    details = []

    for item in income_items:
        income_type = item.get("type")
        if income_type == "salary":
            result = assess_salary_income(item)
        elif income_type == "bonus":
            result = assess_bonus_income(item)
        elif income_type == "housing":
            result = assess_housing_benefit(item)
        elif income_type == "pension":
            result = assess_pension_income(item)
        elif income_type == "stock_option":
            result = assess_stock_option_income(item)
        elif income_type == "share_award":
            result = assess_share_award_income(item)
        elif income_type == "lump_sum":
            result = assess_lump_sum_income(item)
        elif income_type == "fringe_benefit":
            result = assess_fringe_benefit(item)
        else:
            result = {"taxable": False, "taxable_amount": 0, "reason": f"Unknown income type: {income_type}"}

        if result.get("taxable", False):
            total += result.get("taxable_amount", 0)

        result["type"] = income_type
        details.append(result)

    return {"total_income": round(total, 2), "details": details}


Final

In [60]:
def calculate_final_salaries_tax(user_data: dict) -> dict:
    """
    Main function to calculate final salaries tax payable for a taxpayer,
    including total income, deductions, allowances, and one-off rebate.

    Input: user_data = {
        "income_items": [...],
        "deductions": {...},
        "personal": {...},
        "year_of_assessment": int (e.g. 2024)
    }

    Returns: dict with breakdown and tax result
    """

    # 1. 汇总收入
    income_result = calculate_total_income(user_data.get("income_items", []))
    total_income = income_result["total_income"]

    # 2. 汇总扣除
    deduction_result = calculate_total_deductions(user_data.get("deductions", {}))
    total_deductions = deduction_result["total_deduction"]

    # 3. 汇总免税额
    allowance_result = calculate_total_allowances(user_data.get("personal", {}))
    total_allowances = allowance_result["total_allowance"]

    # 4. 计算净应课税收入
    net_chargeable_income = max(0, total_income - total_deductions - total_allowances)

    # 5. 累进税计算（2%、6%、10%、14%、17%）
    brackets = [50000, 50000, 50000, 50000]
    rates = [0.02, 0.06, 0.10, 0.14, 0.17]
    remaining = net_chargeable_income
    progressive_tax = 0

    for i, b in enumerate(brackets):
        step = min(b, remaining)
        progressive_tax += step * rates[i]
        remaining -= step
        if remaining <= 0:
            break
    if remaining > 0:
        progressive_tax += remaining * rates[-1]

    # 6. 标准税（15%）
    standard_tax = max(0, (total_income - total_deductions) * 0.15)

    # 7. 选较低税额
    final_tax = min(progressive_tax, standard_tax)
    method = "progressive" if final_tax == progressive_tax else "standard"

    # 8. 应用 one-off 减免（如有）
    year = user_data.get("year_of_assessment", 2024)
    rebate_result = apply_one_off_rebate(year, final_tax)
    final_tax_after_rebate = rebate_result["final_tax"]

    # 9. 输出结构
    return {
        "total_income": total_income,
        "total_deductions": total_deductions,
        "total_allowances": total_allowances,
        "net_chargeable_income": net_chargeable_income,
        "tax_method_used": method,
        "progressive_tax": round(progressive_tax, 2),
        "standard_tax": round(standard_tax, 2),
        "tax_before_rebate": round(final_tax, 2),
        "final_tax_payable": round(final_tax_after_rebate, 2),
        "one_off_rebate": rebate_result,
        "breakdown": {
            "income": income_result["details"],
            "deductions": deduction_result["details"],
            "allowances": allowance_result["details"]
        }
    }


In [61]:
result = calculate_final_salaries_tax({
    "income_items": [
        {"type": "salary", "amount": 500000},
        {"type": "bonus", "amount": 50000},
        {"type": "housing", "income": 500000, "nrv_ratio": 0.1, "rent_suffered": 20000, "rateable_value": 40000}
    ],
    "deductions": {
        "self_education": {"amount": 80000, "is_qualified_course": True},
        "retirement": {"amount": 18000, "is_mpf": True},
        "ars_expense": {"amount": 90000, "is_qualified": True}
    },
    "personal": {
        "is_married": False,
        "num_children": 0,
        "num_parents": 0,
        "is_disabled": False
    },
    "year_of_assessment": 2024
})


NameError: name 'assess_salary_income' is not defined